In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import re
import config #contains hidden api key


## VOA column headers

I can't find the VOA column headers so have had to take them from the data description pdf [here](https://voaratinglists.blob.core.windows.net/html/documents/2017%20Compiled%20List%20and%20SMV%20Data%20Specification.pdf)

In [3]:
VOA_headers_raw= ["Incrementing Entry Number", "Billing Authority Code", "NDR Community Code", 
 "BA Reference Number", "Primary And Secondary Description Code", "Primary Description Text",
"Unique Address Reference Number UARN", "Full Property Identifier", "Firms Name", "Number Or Name",
"Street", "Town", "Postal District", "County", "Postcode", "Effective Date", "Composite Indicator",
 "Rateable Value", "Appeal Settlement Code", "Assessment Reference", "List Alteration Date", "SCAT Code And Suffix",
 "Sub Street level 3", "Sub Street level 2", "Sub Street level 1", "Case Number", 
 "Current From Date", "Current To Date", 
]

#set to lower and replace spaces with underscore to turn the names into appropriate column names
VOA_headers = [x.lower().replace(" ", "_") for x in VOA_headers_raw]


Using the attention mechanism dramatically improves the quality of the results, but there are still some substantial issues

In [4]:
voa_businesses =  pd.read_csv('/tf/empty_homes_data/' +
                    'uk-englandwales-ndr-2017-listentries-compiled-epoch-0029-baseline-csv.csv',
                   sep = "*",
                   encoding_errors= 'ignore',
                    header=None,
                   names = VOA_headers[1:],
                    usecols = list(range(1,28))
                   )
voa_businesses['postcode'] = voa_businesses['postcode'].str.upper()

#Create a dataframe that contains the counts of businesses per postcode
postcode_counts_voa = voa_businesses.groupby('postcode').size().reset_index(name = 'business_counts')

<ipython-input-4-0afde0e66eed>:1: DtypeWarning: Columns (1,2,6,12,13,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  voa_businesses =  pd.read_csv('/tf/empty_homes_data/' +


In [5]:
voa_businesses["street"]

0             ABBEYFIELDS
1             ABBEYFIELDS
2             ADAMS CLOSE
3             ADAMS CLOSE
4             ADAMS CLOSE
                ...      
2553594       HAMLET ROAD
2553595    WHITING STREET
2553596     HOLLANDS ROAD
2553597     HOLLANDS ROAD
2553598          FEN BANK
Name: street, Length: 2553599, dtype: object

In [6]:
voa_businesses#.iloc[:, 6:14]

,billing_authority_code,ndr_community_code,ba_reference_number,primary_and_secondary_description_code,primary_description_text,unique_address_reference_number_uarn,full_property_identifier,firms_name,number_or_name,street,...,appeal_settlement_code,assessment_reference,list_alteration_date,scat_code_and_suffix,sub_street_level_3,sub_street_level_2,sub_street_level_1,case_number,current_from_date,current_to_date
0,235,NaN,3000837,MH,SURGERY AND PREMISES,4159370000,"MEDICAL CENTRE, ABBEYFIELDS, ELSTOW, BEDFORD",NaN,MEDICAL CENTRE,ABBEYFIELDS,...,NaN,2.345050e+10,01-JUL-2020,436G,NaN,NaN,NaN,3.427498e+10,01-JUL-2020,NaN
1,235,NaN,3000449,EL,SCHOOL AND PREMISES,8871511000,"ELSTOW SCHOOL, ABBEYFIELDS, ELSTOW, BEDFORD",NaN,ELSTOW SCHOOL,ABBEYFIELDS,...,NaN,2.038070e+10,11-MAR-2019,159G,NaN,NaN,NaN,3.334318e+10,11-MAR-2019,NaN
2,235,NaN,3001255,IF,FACTORY AND PREMISES,287299011,"5, ADAMS CLOSE, KEMPSTON, BEDFORD",NaN,5,ADAMS CLOSE,...,NaN,1.574437e+10,NaN,096G,NaN,NaN,NaN,2.096310e+10,01-APR-2017,NaN
3,235,NaN,1077209,IF3,WORKSHOP AND PREMISES,456757000,"WOBURN HOUSE 3, ADAMS CLOSE, KEMPSTON, BEDFORD",NaN,WOBURN HOUSE 3,ADAMS CLOSE,...,NaN,1.574433e+10,NaN,096G,NaN,NaN,NaN,2.092087e+10,01-APR-2017,NaN
4,235,NaN,1132714,IF,FACTORY AND PREMISES,287049011,"1, ADAMS CLOSE, KEMPSTON, BEDFORD",NaN,1,ADAMS CLOSE,...,NaN,2.145827e+10,06-NOV-2019,096G,NaN,NaN,NaN,3.363070e+10,06-NOV-2019,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553594,3545,NaN,401107480030,CO,OFFICES AND PREMISES,13296010000,"OFFICES 6-8 1ST FLOOR, WEAVERS BUSINESS CENTR...",NaN,OFFICES 6-8 1ST FLOOR,HAMLET ROAD,...,NaN,2.389786e+10,25-MAY-2021,203G,NaN,NaN,WEAVERS BUSINESS CENTRE 6,3.534281e+10,25-MAY-2021,NaN
2553595,3545,NaN,400718980142,CO,OFFICES AND PREMISES,13296011000,"ROOMS G/H, HOSTE HOUSE REFORMED CHURCH, WHITIN...",NaN,ROOMS G/H,WHITING STREET,...,NaN,2.389786e+10,25-MAY-2021,203G,NaN,NaN,HOSTE HOUSE REFORMED CHURCH,3.534281e+10,25-MAY-2021,NaN
2553596,3545,NaN,401146540116,CO,OFFICE AND PREMISES,13511012000,"UNIT 6, MENTA BUSINESS CENTRE 21-27, HOLLANDS ...",NaN,UNIT 6,HOLLANDS ROAD,...,NaN,2.431176e+10,08-NOV-2021,203G,NaN,NaN,MENTA BUSINESS CENTRE 21-27,3.560966e+10,08-NOV-2021,NaN
2553597,3545,NaN,401146540114,CO,OFFICE AND PREMISES,13511008000,"UNIT 5, MENTA BUSINESS CENTRE 21-27, HOLLANDS ...",NaN,UNIT 5,HOLLANDS ROAD,...,NaN,2.431171e+10,08-NOV-2021,203G,NaN,NaN,MENTA BUSINESS CENTRE 21-27,3.560965e+10,08-NOV-2021,NaN


## Find business counts per postcode

In [3]:
ocod_data =  pd.read_csv('/tf/empty_homes_data/' +
                    'OCOD_FULL_2022_02.csv',
                   encoding_errors= 'ignore').rename(columns = lambda x: x.lower().replace(" ", "_"))
ocod_data['postcode'] = ocod_data['postcode'].str.upper()
ocod_data = ocod_data.dropna(subset = 'property_address')
ocod_data.reset_index(inplace = True, drop = True)
ocod_data['property_address'] = ocod_data['property_address'].str.lower()
#ocod_data['postcode'] = ocod_data['postcode'].str.replace(" ", "")                 

<ipython-input-3-10b198a5ff45>:1: DtypeWarning: Columns (24,28,30,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  ocod_data =  pd.read_csv('/tf/empty_homes_data/' +


In [5]:

flatregex = r"(the.+)?(flat|apartment|penthouse)(s)?(\s(no)?(\S+?)(\s|,)(\s?(to|-)\s?.+?(\s|,))?|,)"
road_regex = r"(road|street|lane|way|avenue|close|drive|hill|place|terrace|crescent|gardens|square|walk|grove|mews|row)(\s(east|west|north|south))?"

flatregex2 = r"(mansions|villas|court)(?!(\s"+road_regex+"))"
    
#flatregex2 = r"^(^[0-9]+([a-z])?)(( \\w+)('s)?)+ (court|villas|mansions)"
ocod_data['flat_tag'] = ocod_data['property_address'].str.contains(flatregex + '|'+flatregex2, case = False)

commercial_regex = "(cinema)|(hotel)|(office)|(pub)|(business)|(cafe)|(^shop)|( shop)|(restaurant)|unit"
ocod_data['business_tag'] = ocod_data['property_address'].str.contains(commercial_regex, case = False)

ocod_data['carpark_tag'] = ocod_data['property_address'].str.contains("^car |^park|^garage", case = False)
#ocod_data['garage_tag'] = ocod_data['property_address'].str.contains("", case = False)

ocod_data['land_tag'] = ocod_data['property_address'].str.contains("^(land|plot)", case = False)

ocod_data['airspace_tag'] = ocod_data['property_address'].str.contains("^airspace", case = False)

named_house_regex = "^([a-z]+)( [a-z]+)* (house|court|farm|villa|cottage|lodge)(s)?"
ocod_data['named_house_tag'] = ocod_data['property_address'].str.contains(named_house_regex, case = False)

ocod_data['comma_counts'] = ocod_data['property_address'].str.count(",")


#this regex indicates that there are multple properties in the format "xx to yy" or "xx-yy" way of writing things
xx_to_yy_regex = r'(\d)(\s?(to|-)\s?)(\d)'#'(\\s(no)?(\\S+?)(\\s|,)(\\s?(to|-)\\s?.+?(\\s|,))?|,)'
odds_evens_regex = r'(\s|\()?(odd|even)'
plural_dwelling_regex = r'(flats|apartments|penthouses|houses)'
ocod_data['xx_to_yy'] = ocod_data['property_address'].str.contains(xx_to_yy_regex )
ocod_data['odds_evens'] = ocod_data['property_address'].str.contains(odds_evens_regex)
ocod_data['plural_dwelling'] = ocod_data['property_address'].str.contains(plural_dwelling_regex)


<ipython-input-5-d876b3f7db47>:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ocod_data['flat_tag'] = ocod_data['property_address'].str.contains(flatregex + '|'+flatregex2, case = False)
<ipython-input-5-d876b3f7db47>:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ocod_data['business_tag'] = ocod_data['property_address'].str.contains(commercial_regex, case = False)
<ipython-input-5-d876b3f7db47>:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ocod_data['land_tag'] = ocod_data['property_address'].str.contains("^(land|plot)", case = False)
<ipython-input-5-d876b3f7db47>:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ocod_data['n

In [ ]:
ocod_data[['property_address', 'flat_tag']].rename(columns = {'property_address':'text'}).to_csv('/tf/empty_homes_data/property_address_only.csv')

In [15]:
test_regex = r"((\d+)(\s?(to|-)\s?)(\d+)\b)(?!(\s([a-z])+('s)?)+(\s(road|street|lane|wayavenue|close|drive|hill|place|terrace|crescent|gardens|square|walk|grove|mews|row)))"
ocod_data['property_address'].str.contains(test_regex).sum()

<ipython-input-15-97063ddfa5ee>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ocod_data['property_address'].str.contains(test_regex).sum()


4255

In [6]:
 ocod_data[ocod_data['property_address'].str.contains('excluding', case = False)==True]


,title_number,tenure,property_address,district,county,region,postcode,multiple_address_indicator,price_paid,proprietor_name_(1),...,flat_tag,business_tag,carpark_tag,land_tag,airspace_tag,named_house_tag,comma_counts,xx_to_yy,odds_evens,plural_dwelling
38,SYK594298,Freehold,"1-25 (excluding 13) acorn way, sunnyside, roth...",ROTHERHAM,SOUTH YORKSHIRE,YORKS AND HUMBER,S66 3XH,N,NaN,ABACUS LAND 1 (HOLDCO 1) LIMITED,...,False,False,False,False,False,False,2,True,False,False
1901,NGL421367,Freehold,the open ended garage in the basement of the b...,CITY OF WESTMINSTER,GREATER LONDON,GREATER LONDON,NaN,N,NaN,38 TP LIMITED,...,False,False,False,False,False,False,1,False,False,False
5590,HD416123,Freehold,"1 to 16 (excluding 13) barge house, evans whar...",DACORUM,HERTFORDSHIRE,SOUTH EAST,HP3 9WE,N,NaN,ADRIATIC LAND 11 LIMITED,...,False,False,False,False,False,False,2,True,False,False
7758,236218,Freehold,17 to 27 (odd)(excluding 19) hill street and 3...,CITY OF WESTMINSTER,GREATER LONDON,GREATER LONDON,NaN,N,NaN,BERKELEY SQUARE HOLDINGS LIMITED,...,False,False,False,False,False,False,6,True,True,False
19975,NGL585687,Leasehold,parts of 71 queen victoria street as more part...,CITY OF LONDON,GREATER LONDON,GREATER LONDON,NaN,N,NaN,QUEEN V PROPERTY UK I LTD,...,False,False,False,False,False,False,0,False,False,False
29454,NGL881926,Leasehold,10 old burlington street excluding parts of th...,CITY OF WESTMINSTER,GREATER LONDON,GREATER LONDON,NaN,N,NaN,10-13 OLD BURLINGTON STREET LIMITED,...,False,False,False,False,False,False,0,False,False,False
32870,SYK594213,Leasehold,1 to 69 (odd but excluding 13) and 2 to 44 (ev...,SHEFFIELD,SOUTH YORKSHIRE,YORKS AND HUMBER,NaN,N,3902021.0,ADRIATIC LAND 3 LIMITED,...,False,False,False,False,False,False,0,True,True,False
42138,EGL375156,Leasehold,"floor 36 (excluding common parts), 1 canada sq...",TOWER HAMLETS,GREATER LONDON,GREATER LONDON,E14 5DX,N,NaN,EULER HERMES CREDIT INSURANCE BELGIUM SA (NV),...,False,False,False,False,False,False,2,False,False,False
42182,AGL470670,Leasehold,airspace from a height of 30.300 metres above ...,HACKNEY,GREATER LONDON,GREATER LONDON,N1 6AZ,N,NaN,O & R LIMITED,...,False,False,False,False,True,False,3,False,False,False
42273,BM252802,Leasehold,two parcels of land forming part of great west...,BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,SOUTH EAST,NaN,N,NaN,ROCKSPRING UK VALUE 2 AYLESBURY (JERSEY) LIMITED,...,False,False,False,False,False,False,3,False,False,False


In [14]:
ocod_data.columns

Index(['title_number', 'tenure', 'property_address', 'district', 'county',
       'region', 'postcode', 'multiple_address_indicator', 'price_paid',
       'proprietor_name_(1)', 'company_registration_no._(1)',
       'proprietorship_category_(1)', 'country_incorporated_(1)',
       'proprietor_(1)_address_(1)', 'proprietor_(1)_address_(2)',
       'proprietor_(1)_address_(3)', 'proprietor_name_(2)',
       'company_registration_no._(2)', 'proprietorship_category_(2)',
       'country_incorporated_(2)', 'proprietor_(2)_address_(1)',
       'proprietor_(2)_address_(2)', 'proprietor_(2)_address_(3)',
       'proprietor_name_(3)', 'company_registration_no._(3)',
       'proprietorship_category_(3)', 'country_incorporated_(3)',
       'proprietor_(3)_address_(1)', 'proprietor_(3)_address_(2)',
       'proprietor_(3)_address_(3)', 'proprietor_name_(4)',
       'company_registration_no._(4)', 'proprietorship_category_(4)',
       'country_incorporated_(4)', 'proprietor_(4)_address_(1)',
     

In [11]:
columns_of_interest = ['title_number', 'tenure', 'property_address', 'district', 'county',
       'region', 'postcode', 'multiple_address_indicator', 'price_paid',
      'flat_tag',
       'business_tag', 'carpark_tag', 'land_tag', 'named_house_tag',
       'comma_counts', 'plural_dwelling', 'xx_to_yy','odds_evens',
       'odds_evens']

In [57]:
#ocod_data.loc[(ocod_data['xx_to_yy']==True) | (ocod_data['plural_dwelling']==True)| (ocod_data['odds_evens']==True),columns_of_interest].to_csv('/tf/empty_homes_data/multi_flat.csv')

In [60]:
ocod_data.groupby(['business_tag' ]).size()

business_tag
False    88624
True      5464
dtype: int64

In [6]:
ocod_data[(ocod_data['property_address'].str.contains("apartments ", case = False)==True) & 
         (ocod_data['property_address'].str.contains(" land", case = False)!=True) &
         (ocod_data['property_address'].str.contains("^land", case = False)!=True)].to_csv("/tf/empty_homes_data/multiple_apartments.csv")

In [12]:
#add in the counts of businesses in that postcode
ocod_data = pd.merge(ocod_data, postcode_counts_voa, on = "postcode", how = "left")
ocod_data["business_counts"] = ocod_data["business_counts"].fillna(0)

In [13]:
#classify the property type using this tree structure. 
#N.B. If you use the conditions in a different order you will get differencet results
#This produces rule based noisey labels. The idea is to have a good enough system that can be improved on
ocod_data['class'] = np.select(
    [
        ocod_data['flat_tag']==True,
        ocod_data['land_tag'] == True,
        ocod_data['carpark_tag'] == True,
        ocod_data['airspace_tag'] == True,
        ocod_data['business_tag'] == True,
        ocod_data['business_counts']==0,
    ], 
    [
        'domestic', 
        'land',
        'carpark',
        'airspace',
        'business',
        'domestic'
    ], 
    default='unknown'
)

In [29]:
#ocod_data[(ocod_data['business_counts']==0 )& (ocod_data['business_tag'])]
ocod_data[(ocod_data['class']=='unknown' )]
ocod_data['property_address'].sample(n=10000).to_csv('/tf/empty_homes_data/offshore_addresses.csv', header = False)

In [14]:
ocod_data.groupby('class').size()

class
airspace     1569
business     4927
carpark      2178
domestic    45368
land        16477
unknown     23569
dtype: int64

# processing multi addresses 

The multi addresses are quite difficult to use. 
This is section is to help process them

In [34]:
 multi_address = ocod_data.loc[:, columns_of_interest][(ocod_data['comma_counts']>5) | (ocod_data['plural_dwelling']==True) | (ocod_data['xx_to_yy']==True) | (ocod_data['odds_evens']==True)]

In [40]:
multi_address[ 'property_address']

13       2-24 (even) Miller Way, 15-25 Hammonds Drive (...
38       1-25 (excluding 13) Acorn Way, Sunnyside, Roth...
46       70 To 98 (even), Weavermill Park, Ashton In Ma...
48       1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 16, 18,...
56       2 and 4 Colliers Grove, 16 Gadfield Grove, 11,...
                               ...                        
94008                   Units, 1-5 Brewery Road, Hoddesdon
94030                           1-21 Gatcombe Mews, Ealing
94048    50 to 55 (inclusive), 62 to 73 (inclusive) and...
94050    14-50 (even), Windermere Drive, Doncaster (DN4...
94059    Sixth Floor Premises, 6-8 Tokenhouse Yard, Lon...
Name: property_address, Length: 9479, dtype: object

In [41]:
multi_address['property_address'].to_list()

['2-24 (even) Miller Way, 15-25 Hammonds Drive (odd) and 1-19 Rudd Close (odd) Fengate, Peterborough',
 '1-25 (excluding 13) Acorn Way, Sunnyside, Rotherham (S66 3XH)',
 '70 To 98 (even), Weavermill Park, Ashton In Makerfield, Wigan (WN4 9EZ)',
 '1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 16, 18, 19, 21 and 22 Heath House Close, Lowton, and 106 and 108 Stone Cross Lane North, Lowton, Warrington',
 '2 and 4 Colliers Grove, 16 Gadfield Grove, 11, 12, 16, 19, 21, 22, 24-30, 32, 33, 35-37 and 41 Albion Close, Atherton, Manchester',
 '2 to 4 and 6 to 11 (inclusive), Junction Close, Blackrod, Bolton (BL6 5GN)',
 '3 and 5 Bloomsbury Crescent, 2, 4, 7, 9, 11, 12 and 14 Burlington Drive and 2, 3, 5, 14 and 16 Windsor Gardens, Bolton',
 '59 and 71 Lawnhurst Avenue, Apartment 8, 240 Lawnhurst Avenue, Apartments 3, 8 and 13, 242 Lawnhurst Avenue, Apartment 6, 244 Lawnhurst Avenue, Apartments 4 and 9, 246 Lawnhurst Avenue and Apartment 13, 248 Lawnhurst Avenue, Wythenshawe, Manchester',
 '6, 15, 16, 17

In [79]:
single_address = multi_address['property_address'].to_list()[0]
print(single_address)



2-24 (even) Miller Way, 15-25 Hammonds Drive (odd) and 1-19 Rudd Close (odd) Fengate, Peterborough


In [5]:
ocod_data['property_address'].str.contains("fields", case = False).sum()


292

In [ ]:
road_name_regex = "(( \w+)('s)?)+ (road|lane|street|way|avenue|close|drive|hill|place|terrace|crescent|gardens|square|walk|grove|mews|row|centre|view|cottages|yard|bank|rise|gate|parade|common|cross|village|mill|station|corner|croft|meadow|fields|quay|wood|field|fold|passage|( \w+)+(way|gate,)|knightsbridge)"

In [183]:
search_pattern = re.compile("^\d+\d*")
print(search_pattern)
match = [re.search(search_pattern, x) for x in ocod_data['property_address'].str.lower().to_list()]



re.compile('^\\d+\\d*')


In [184]:
i = 0
for x in match:
    i= i + int(x != None)
i

34130

In [246]:
ocod_data['property_address'][np.array(match)!=None]

NameError: name 'match' is not defined

In [6]:
ocod_data[['property_address', 'flat_tag']].rename(columns = {'property_address':'text'}).to_csv('/tf/empty_homes_data/property_address_only.csv')


In [272]:
ocod_data[['property_address', 'flat_tag']].shape

(94088, 2)

In [30]:
#class counts after first classifying attempt
class_by_district = ocod_data.groupby(['class', 'district']).size()
class_by_district.to_csv ('class_by_district.csv')

In [31]:

postcode_counts_ocod = ocod_data[(ocod_data['class']=='unknown' )].groupby('postcode').size().reset_index(name = "ocod_counts")

In [32]:
postcode_matches = pd.merge(postcode_counts_voa, postcode_counts_ocod, on = "postcode", how = "outer")
postcode_matches['post_area'] = postcode_matches['postcode'].str.extract('([^\d]+)')
postcode_matches['non_commercial'] = postcode_matches['business_counts'].isna() & postcode_matches['ocod_counts'].notna()

In [33]:

post_area_list = postcode_matches[postcode_matches['post_area'].notna()]['post_area'].tolist()
post_area_list = np.unique(post_area_list)
post_area_list

array(['AL', 'B', 'BA', 'BB', 'BD', 'BH', 'BL', 'BN', 'BR', 'BS', 'C',
       'CA', 'CB', 'CF', 'CH', 'CM', 'CN', 'CO', 'CR', 'CT', 'CV', 'CW',
       'DA', 'DE', 'DG', 'DH', 'DL', 'DN', 'DT', 'DY', 'E', 'EC', 'EN',
       'EX', 'FY', 'G', 'GL', 'GU', 'HA', 'HD', 'HG', 'HP', 'HR', 'HU',
       'HX', 'IG', 'IP', 'KT', 'L', 'LA', 'LD', 'LE', 'LL', 'LN', 'LS',
       'LU', 'M', 'MD', 'ME', 'MK', 'N', 'NE', 'NG', 'NH', 'NN', 'NP',
       'NR', 'NW', 'OL', 'OX', 'PE', 'PL', 'PO', 'PR', 'RG', 'RH', 'RM',
       'S', 'SA', 'SE', 'SG', 'SH', 'SK', 'SL', 'SM', 'SN', 'SO', 'SP',
       'SR', 'SS', 'ST', 'SW', 'SY', 'TA', 'TD', 'TF', 'TH', 'TN', 'TQ',
       'TR', 'TS', 'TW', 'UB', 'W', 'WA', 'WC', 'WD', 'WF', 'WN', 'WR',
       'WS', 'WV', 'YO'], dtype='<U2')

In [34]:
postcode_matches['conservative_domestic'] = postcode_matches['ocod_counts'].fillna(0)-postcode_matches['business_counts'].fillna(0)
postcode_matches['conservative_domestic'] = np.where(postcode_matches['conservative_domestic']<0, 0 , postcode_matches['conservative_domestic'])


In [35]:
postcode_matches.sort_values(by=['ocod_counts'], ascending=False)

,postcode,business_counts,ocod_counts,post_area,non_commercial,conservative_domestic
350614,SE1 7UT,19,577.0,SE,False,558.0
218794,LS7 1NJ,1,178.0,LS,False,177.0
122879,E1 0HJ,19,78.0,E,False,59.0
350601,SE1 7RY,4,76.0,SE,False,72.0
199845,LE3 5HU,1,41.0,LE,False,40.0
...,...,...,...,...,...,...
470045,YO8 9TQ,1,NaN,YO,False,0.0
470046,YO8 9UB,2,NaN,YO,False,0.0
470047,YO8 9UG,5,NaN,YO,False,0.0
470048,YO8 9UH,2,NaN,YO,False,0.0


In [36]:
#number of OCOD with no business in postcode
postcode_matches[postcode_matches['business_counts'].isna() & postcode_matches['ocod_counts'].notna()][['ocod_counts']].sum()

ocod_counts    0.0
dtype: float64

In [37]:
#number of OCOD with with business in postcode
postcode_matches[postcode_matches['business_counts'].notna() & postcode_matches['ocod_counts'].notna()][['ocod_counts']].sum()

ocod_counts    25966.0
dtype: float64

In [38]:
print('Total ocod ' + str(postcode_matches['ocod_counts'].sum() )+
    ' Conservative domestic counts ' + str(postcode_matches['conservative_domestic'].sum()))

Total ocod 25966.0 Conservative domestic counts 2728.0


In [39]:
ocod_data.groupby('proprietor_name_(1)').agg('size')

proprietor_name_(1)
(CR) CONSTABLE REAL B.V.         1
1 BL EC2 LTD                     1
1 BRIDGE STREET LIMITED          1
1 CAPITAL LIMITED                1
1 CHESHAM CLOSE LIMITED          1
                                ..
ZUSHCU PROPERTIES PVT LIMITED    4
ZUSHCU PROPERTIES PVT LTD        1
ZWAN HOLDINGS LIMITED            1
ZWIKKE LIMITED                   3
ZYLAND INTERNATIONAL LIMITED     4
Length: 31809, dtype: int64

In [40]:
address_list = ocod_data['property_address'].tolist()[0:100]
parsed_address_list_att = address_parser_att(address_list)
address_errors = pd.DataFrame([x.to_dict() for x in parsed_address_list_att]).drop(
    ["Orientation", "GeneralDelivery", "EOS"], axis  = 1)
address_errors["address"] = address_list
address_errors

NameError: name 'address_parser_att' is not defined

In [54]:
import io
import zipfile


post_area_id = 'AL'

target_post_area = "Data/multi_csv/ONSPD_NOV_2021_UK_" + post_area_id +".csv"

with zipfile.ZipFile("/tf/empty_homes_data/" + "ONSPD_NOV_2021_UK.zip") as zf:
    with io.TextIOWrapper(zf.open(target_post_area), encoding="utf-8") as f:
        ONSPD = pd.read_csv(f)

temp_postcode_matches = postcode_matches[postcode_matches['post_area']==post_area_id]

temp_postcode_matches   


,postcode,business_counts,ocod_counts,post_area
42,AL1 1AW,1.0,NaN,AL
43,AL1 1BX,7.0,NaN,AL
44,AL1 1BY,4.0,NaN,AL
45,AL1 1BZ,21.0,NaN,AL
46,AL1 1DQ,2.0,NaN,AL
...,...,...,...,...
470107,AL8 6UB,NaN,1.0,AL
470108,AL9 5RH,NaN,1.0,AL
470109,AL9 7QG,NaN,1.0,AL
470110,AL9 7RX,NaN,1.0,AL


In [5]:
import json
# Opening JSON file
#f =open("/tf/empty_homes_data/exported_results_2022-02-17T08_14_04.244Z.json") #download button
f =open("/tf/empty_homes_data/exported_results_2022-02-28T15_47_31.229Z.json")  #aggregate and download button

#exported_results_2022-02-21T13_58_40.285Z.json
# returns JSON object as 
# a dictionary
data = json.load(f)
   
# Iterating through the json
# list
#for i in data['emp_details']:
#    print(i)
   
# Closing file
#f.close()


I need to create a dataset which has the following structure

[("Tokyo Tower is 333m tall.", [(0, 11, "BUILDING")])]

That is each observation is an element of a list

each element is a two part tuple where the first part is a text string and the second part is a list
each element of this list is a three part tuple where
part1 the beggining of the span
part 2 the end of the span
part three the class of the span
  
This is shown on the spacy docs https://spacy.io/usage/training#training-data

In [6]:

datapoint_id_list = [x['datapointId'] for x in data]
print(max(datapoint_id_list))

len(set(datapoint_id_list))

94087


93442

In [8]:


def is_overlap_function(a,b):
    #identifies any overlapping tags.
    #takes two dictionaries and outputs a logical value identifying if the tags overlap
     start_overlap = (b['start'] >= a['start']) & (b['start'] <= a['end']) 
     end_overlap = (b['end'] >= a['start']) & (b['end'] <= a['end']) 
     return (start_overlap | end_overlap)

    
def remove_overlapping_spans(list_of_labels_dict):
    
    
    span_length = [x['end']-x['start'] for x in list_of_labels_dict]
    #produces a list of indices indicating the first element of a list in a pair that overlaps
    #this means if (start, end) [(0,3), (4,7), ),(5,9)], the value returned would be 1 because indices 1 and 2 overlap.
    is_overlap_list = [x for x in [*range(0, len(list_of_labels_dict)-1)] if
                  is_overlap_function(list_of_labels_dict[x],list_of_labels_dict[x+1])]

    #identify the index of the list to be deleted
    del_list =[]
    for i in is_overlap_list:
         pair_overlaps = is_overlap_function(list_of_labels_dict[i],list_of_labels_dict[i+1])
         if span_length[i] > span_length[i+1]:
            del_list = del_list + [i+1]
         else:
            del_list = del_list + [i]
        

    no_overlaps_list = [i for j, i in enumerate(list_of_labels_dict) if j not in del_list]
    return no_overlaps_list


def remove_overlapping_spans2(list_of_labels_dict):
    #This function iterates through the list pairwise checking to see if there are overlaps
    #this function builds on the previous version which checked for overlaps in one go
    #but didn't consider that a single entity might overlap several subsequent entities
    #example A (start = 0, end = 20), B (start = 3, end = 10), C (start = 12, end = 20)
    #A overlaps B and C even though B and C do not overlap, as a result only A should remain
    
    i = 0
    j = i+1
    while j < len(list_of_labels_dict):
        
        
        #check if there is overlap
        pair_overlaps = is_overlap_function(list_of_labels_dict[i],list_of_labels_dict[j])
        if pair_overlaps:
            #get index of smallest span
            span_a = list_of_labels_dict[i]['end'] - list_of_labels_dict[i]['start']
            span_b = list_of_labels_dict[j]['end'] - list_of_labels_dict[j]['start']

            if span_a > span_b:
                list_of_labels_dict.pop(j)
            else:
                list_of_labels_dict.pop(i)
        else:
            #i and j are only updated here as if either is removed the indexing changed such that the original i and j values now represent differnt elements
            #only when there is no overlap should the indexes advance
            i = i +1
            j = i +1
        
    return list_of_labels_dict


In [10]:


import random
import operator #used for sorting the label dictionaries by start point. This is the basis for removing overlaps

random.seed(10)
randomlist = random.sample(datapoint_id_list, 9)
#print(randomlist)
#randomlist.pop(54)
data_and_labels = []
data_labels_dict = []

count_it = 0
for i in randomlist:
    count_it += 1
    if count_it % 100 == 0: 
        print('count = {}'.format(count_it))
        
    single_id_index = np.where(np.array(datapoint_id_list)==i)
    ##these labels are in tuple form
   # list_of_labels = [(data[x]['start'], data[x]['end'], data[x]['label']) for x in single_id_index[0].tolist()]
    
    ##these labels are in dictionary form
    list_of_labels_dict = [{'start': data[x]['start'], 'end':data[x]['end'], 'label': data[x]['label']} for x in single_id_index[0].tolist()]
    
    #this inplace sorting using operator orders the dictionary by the start point. ties are automatically broken
    #it required the operator library
    list_of_labels_dict.sort(key=operator.itemgetter('start'))
    
    list_of_labels_dict = remove_overlapping_spans2(list_of_labels_dict)

    #create the NER dataset structure shown on the spacy website
   # data_and_labels = data_and_labels + [ ( ocod_data['property_address'][i], list_of_labels ) ]
    #create a list of dictionaries using a similar structure to save as a json
    data_labels_dict = data_labels_dict + [
        {
            'text' : ocod_data['property_address'][i],
            'labels' : list_of_labels_dict,
            'datapoint_id': i,
        }
    ]


In [28]:
data_labels_dict[4455]


{'text': '49 thornville road, leeds (ls6 1jy)',
 'labels': [{'start': 0, 'end': 2, 'label': 'street_number'},
  {'start': 3, 'end': 18, 'label': 'street_name'},
  {'start': 20, 'end': 25, 'label': 'city'}]}

In [79]:
data

[{'datapointId': 89214,
  'labelId': 7,
  'start': 42,
  'end': 54,
  'text': 'chesterfield',
  'label': 'city',
  'probability': 1},
 {'datapointId': 19385,
  'labelId': 12,
  'start': 3,
  'end': 16,
  'text': 'cannon street',
  'label': 'street_name',
  'probability': 1},
 {'datapointId': 63519,
  'labelId': 11,
  'start': 57,
  'end': 64,
  'text': 'w14 8ln',
  'label': 'postcode',
  'probability': 1},
 {'datapointId': 42713,
  'labelId': 13,
  'start': 154,
  'end': 155,
  'text': '1',
  'label': 'street_number',
  'probability': 1},
 {'datapointId': 67716,
  'labelId': 11,
  'start': 41,
  'end': 48,
  'text': 'n17 9eu',
  'label': 'postcode',
  'probability': 1},
 {'datapointId': 59895,
  'labelId': 13,
  'start': 23,
  'end': 28,
  'text': '57-59',
  'label': 'street_number',
  'probability': 1},
 {'datapointId': 51715,
  'labelId': 15,
  'start': 0,
  'end': 4,
  'text': 'land',
  'label': 'unit_type',
  'probability': 1},
 {'datapointId': 14514,
  'labelId': 12,
  'start': 3,

In [15]:
data_labels_dict2 = random.sample(data_labels_dict, 9999) #data_labels_dict
#del_list = [1160,2652,3015,3861,3897,4455, 4485,4859]
del_list = []
data_labels_dict2  = [i for j, i in enumerate(data_labels_dict2) if j not in del_list]
print(data_labels_dict[1160])
print(data_labels_dict[1160]['text'][19:48])
print(data_labels_dict[1160]['text'][34:48])

{'text': 'flat 3, 300 walworth road, london (se17 2te)', 'labels': [{'start': 0, 'end': 4, 'label': 'unit_type', 'label_text': 'flat'}, {'start': 5, 'end': 6, 'label': 'unit_id', 'label_text': '3'}, {'start': 8, 'end': 11, 'label': 'street_number', 'label_text': '300'}, {'start': 12, 'end': 25, 'label': 'street_name', 'label_text': 'walworth road'}, {'start': 27, 'end': 33, 'label': 'city', 'label_text': 'london'}, {'start': 35, 'end': 43, 'label': 'postcode', 'label_text': 'se17 2te'}], 'datapoint_id': 1169}
h road, london (se17 2te)
(se17 2te)


In [17]:
#data_labels_dict2 =  data_labels_dict

jason_test_data ={
     "name": "28_02_22_1010",
     "description": "example labelling structure",
     "fields": [
         {"name": "text", 
          "data_type": "text"
         },
         {"name": "labels", 
          "data_type": "character_offsets"},
         {"name": "datapoint_id", 
          "data_type": "text"
         }
     ],
     "data": data_labels_dict2
}

import requests 

url = "https://api.humanloop.com/datasets"

# replace payload with your actual dataset...
payload= json.dumps(jason_test_data)
headers = {
  'X-API-Key': config.api_key,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"id":1197,"attempted":9999,"succeeded":9999,"msg":"9999 data points uploaded."}


# Create whole dataset json

In [9]:
import operator
data_and_labels = []
data_labels_dict = []

count_it = 0
for i in set(datapoint_id_list):
    count_it += 1
    if count_it % 100 == 0: 
        print('count = {}'.format(count_it))
        
    single_id_index = np.where(np.array(datapoint_id_list)==i)
    ##these labels are in tuple form
   # list_of_labels = [(data[x]['start'], data[x]['end'], data[x]['label']) for x in single_id_index[0].tolist()]
    
    ##these labels are in dictionary form
    list_of_labels_dict = [{'start': data[x]['start'], 
                            'end':data[x]['end'], 
                            'label': data[x]['label'], 
                            'label_text': data[x]['text'] } for x in single_id_index[0].tolist()]
    
    #this inplace sorting using operator orders the dictionary by the start point. ties are automatically broken
    #it required the operator library
    list_of_labels_dict.sort(key=operator.itemgetter('start'))
    
    list_of_labels_dict = remove_overlapping_spans2(list_of_labels_dict)

    #create the NER dataset structure shown on the spacy website
   # data_and_labels = data_and_labels + [ ( ocod_data['property_address'][i], list_of_labels ) ]
    #create a list of dictionaries using a similar structure to save as a json
    data_labels_dict = data_labels_dict + [
        {
            'text' : ocod_data['property_address'][i],
            'labels' : list_of_labels_dict,
            'datapoint_id': i,
        }
    ]

count = 100
count = 200
count = 300
count = 400
count = 500
count = 600
count = 700
count = 800
count = 900
count = 1000
count = 1100
count = 1200
count = 1300
count = 1400
count = 1500
count = 1600
count = 1700
count = 1800
count = 1900
count = 2000
count = 2100
count = 2200
count = 2300
count = 2400
count = 2500
count = 2600
count = 2700
count = 2800
count = 2900
count = 3000
count = 3100
count = 3200
count = 3300
count = 3400
count = 3500
count = 3600
count = 3700
count = 3800
count = 3900
count = 4000
count = 4100
count = 4200
count = 4300
count = 4400
count = 4500
count = 4600
count = 4700
count = 4800
count = 4900
count = 5000
count = 5100
count = 5200
count = 5300
count = 5400
count = 5500
count = 5600
count = 5700
count = 5800
count = 5900
count = 6000
count = 6100
count = 6200
count = 6300
count = 6400
count = 6500
count = 6600
count = 6700
count = 6800
count = 6900
count = 7000
count = 7100
count = 7200
count = 7300
count = 7400
count = 7500
count = 7600
count = 7700
count = 

count = 59400
count = 59500
count = 59600
count = 59700
count = 59800
count = 59900
count = 60000
count = 60100
count = 60200
count = 60300
count = 60400
count = 60500
count = 60600
count = 60700
count = 60800
count = 60900
count = 61000
count = 61100
count = 61200
count = 61300
count = 61400
count = 61500
count = 61600
count = 61700
count = 61800
count = 61900
count = 62000
count = 62100
count = 62200
count = 62300
count = 62400
count = 62500
count = 62600
count = 62700
count = 62800
count = 62900
count = 63000
count = 63100
count = 63200
count = 63300
count = 63400
count = 63500
count = 63600
count = 63700
count = 63800
count = 63900
count = 64000
count = 64100
count = 64200
count = 64300
count = 64400
count = 64500
count = 64600
count = 64700
count = 64800
count = 64900
count = 65000
count = 65100
count = 65200
count = 65300
count = 65400
count = 65500
count = 65600
count = 65700
count = 65800
count = 65900
count = 66000
count = 66100
count = 66200
count = 66300
count = 66400
count 

In [10]:

with open('/tf/empty_homes_data/full_dataset_no_overlaps.json', 'w') as f:
    json.dump(data_labels_dict, f)

In [13]:
data_labels_dict[0]

{'text': 'westleigh lodge care home, nel pan lane, leigh (wn7 5jt)',
 'labels': [{'start': 27,
   'end': 39,
   'label': 'street_name',
   'label_text': 'nel pan lane'},
  {'start': 41, 'end': 46, 'label': 'city', 'label_text': 'leigh'},
  {'start': 48, 'end': 55, 'label': 'postcode', 'label_text': 'wn7 5jt'}],
 'datapoint_id': 0}

In [73]:

histo_thing = all_entities.groupby('datapoint_id').size()

#pd.DataFrame({'size' : histo_thing}).groupby('size').size()